In [53]:
import os, csv, time, random, re, json, requests
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, timezone

DATA_DIR = 'data/lab'
RAW_API_DIR = os.path.join(DATA_DIR, 'raw_api')
RAW_HTML_DIR = os.path.join(DATA_DIR, 'raw_html')
for d in [DATA_DIR, RAW_API_DIR, RAW_HTML_DIR]:
    os.makedirs(d, exist_ok=True)

HEADERS = {"User-Agent": "MADS-LLM-2026-Project (+https://example.edu)"}

def sleep_politely(sleep_range = (0.2,0.5)):
    SLEEP_RANGE = sleep_range
    time.sleep(random.uniform(*SLEEP_RANGE))

In [54]:
# Authentication using app password. OpenAI. (2023). ChatGPT (Mar 14 version) [Large language model]. https://chat.openai.com/chat. In-text citations are simply (OpenAI, 2023) or OpenAI (2023).
pds = "https://bsky.social/xrpc/com.atproto.server.createSession"
payload = {"identifier": "celinavelazquez.bsky.social", "password": "nv22-lumi-pofs-wui7"}

resp = requests.post(pds, json=payload, timeout=60)
resp.raise_for_status()
session = resp.json()
access_jwt = session["accessJwt"]

HEADERS = {"Authorization": f"Bearer {access_jwt}"}

# Refresh token
def refresh_token():
    global access_jwt, refresh_jwt, HEADERS
    r = requests.post(
        "https://bsky.social/xrpc/com.atproto.server.refreshSession",
        headers={"Authorization": f"Bearer {refresh_jwt}"},
        timeout=60
    )
    r.raise_for_status()
    session = r.json()
    access_jwt = session["accessJwt"]
    refresh_jwt = session["refreshJwt"]
    HEADERS = {"Authorization": f"Bearer {access_jwt}"}


In [34]:
# Get posts with the search term "politics"
TOPIC = "politics"
BASE = "https://bsky.social/xrpc/app.bsky.feed.searchPosts"
since_dt = datetime.now(timezone.utc) - timedelta(days=30)
since_str = since_dt.strftime("%Y-%m-%dT%H:%M:%SZ")
params = {"q": TOPIC, "limit":100, "since":since_str, "lang":"en"}

# Pagination. OpenAI. (2023). ChatGPT (Mar 14 version) [Large language model]. https://chat.openai.com/chat. In-text citations are simply (OpenAI, 2023) or OpenAI (2023).
# Pagination State
all_posts = []
cursor = None

#Pagination Loop
while True:
    page_params = params.copy()
    if cursor:
        page_params["cursor"] = cursor

    r = requests.get(BASE, params=page_params, headers=HEADERS, timeout=60)
    print(r.status_code, r.text[:200])
    r.raise_for_status()

    data = r.json()
    posts = data.get('posts', [])
    all_posts.extend(posts)

    cursor = data.get("cursor")
    if not cursor or not posts:
        break

sleep_politely()

len(all_posts)

200 {"posts":[{"uri":"at://did:plc:qjhk7qh3nmhc5jqylpo4laox/app.bsky.feed.post/3meiaakqy5s2w","cid":"bafyreib5py2kjsamjgzh2jnsisdqlvgujzzhn57ryzchcqbhtel5dbedxm","author":{"did":"did:plc:qjhk7qh3nmhc5jqyl
200 {"posts":[{"uri":"at://did:plc:iw7f7zll6xwb4ulmzjhgnrzg/app.bsky.feed.post/3mei5venjy52n","cid":"bafyreihdccp3u7mtwv6ko4go3onpjse6hzsutsrw7wevflosdeo6rmyq6a","author":{"did":"did:plc:iw7f7zll6xwb4ulmz
200 {"posts":[{"uri":"at://did:plc:3lcnf4bx64p5xzvnspryngms/app.bsky.feed.post/3mei4hxz4os2u","cid":"bafyreif2j6unpdvh3x4wdusewbpguam6idrmmvfpp7q4esg5kg3btgr2ei","author":{"did":"did:plc:3lcnf4bx64p5xzvns
200 {"posts":[{"uri":"at://did:plc:vlpy6zuqqum5tumv7b6dw5fp/app.bsky.feed.post/3mei2vplyxs25","cid":"bafyreid6l2onzomcrcaicohepsy4q3oeqxbjfqj6falqnmskkqy5oeslui","author":{"did":"did:plc:vlpy6zuqqum5tumv7
200 {"posts":[{"uri":"at://did:plc:etv6lxbwvhtiby7izetzea3x/app.bsky.feed.post/3mehzg5wxac2n","cid":"bafyreicaug5jrsedqxiqs4ehnscaoveuu2umwv3bkn6admmpo5g76m24ui","author":{"did":"d

9813

In [ ]:
# flatten json
# ALL RECORDS
df = pd.json_normalize(all_posts)

# TESTING - SUBSET of records
#df_TEST = pd.json_normalize(all_posts).iloc[:10].reset_index(drop=True)

In [ ]:
# save to CSV file
df.to_csv(r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_posts.csv")

# TESTING
#df_TEST.to_csv(r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_posts_TEST.csv")

NameError: name 'df_TEST' is not defined

In [ ]:
# filter to posts with >=10 replies
df_10plus = df[df["replyCount"] >= 10].copy()

# Export to CSV
out_csv = r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bluesky_main.csv"

df_10plus.to_csv(out_csv, index=False)

df_10plus.shape


(89, 360)

In [ ]:
# get comments of post in JSONL format

BASE = "https://bsky.social/xrpc/app.bsky.feed.getPostThread"

missing_uris = []
failed_uris = []

out_jsonl = r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_threads.jsonl"

batch_size = 100
uris = df_10plus["uri"].dropna().astype(str).str.strip().reset_index(drop=True)

# Resume set from JSONL
done_uris = set()
if os.path.exists(out_jsonl) and os.path.getsize(out_jsonl) > 0:
    with open(out_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                u = obj.get("uri")
                if u:
                    done_uris.add(str(u))
            except json.JSONDecodeError:
                continue

# Open once for appending (faster than opening per row)
with open(out_jsonl, "a", encoding="utf-8") as jf:
    for start in range(0, len(uris), batch_size):
        batch = uris.iloc[start:start + batch_size]

        for uri in batch:
            if uri in done_uris:
                continue

            try:
                r_t = requests.get(
                    BASE,
                    params={"uri": uri},
                    headers=HEADERS,
                    timeout=60
                )

                # Handle known 400 errors once (no retries)
                if r_t.status_code == 400:
                    err = r_t.json()
                    if err.get("error") == "ExpiredToken":
                        refresh_token()
                        # one more attempt only after refresh, still no retry loop
                        r_t = requests.get(
                            BASE,
                            params={"uri": uri},
                            headers=HEADERS,
                            timeout=20
                        )

                    if r_t.status_code == 400:
                        err = r_t.json()
                        if err.get("error") == "NotFound":
                            missing_uris.append(uri)
                            continue

                # Treat any non-200 as a failure (no retries)
                if r_t.status_code != 200:
                    failed_uris.append(uri)
                    continue

                thread = r_t.json().get("thread")
                if thread is None:
                    failed_uris.append(uri)
                    continue

                jf.write(json.dumps({"uri": uri, "thread": thread}, ensure_ascii=False) + "\n")
                done_uris.add(uri)

            except requests.exceptions.Timeout:
                failed_uris.append(uri)
                continue
            except Exception:
                failed_uris.append(uri)
                continue

len(done_uris), len(missing_uris), len(failed_uris)


(5831, 0, 0)

In [58]:
jsonl_path = r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_threads.jsonl"

records = []

with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        records.append({
            "uri": obj["uri"],
            # keep raw JSON as a string for CSV safety
            "thread_json": json.dumps(obj["thread"], ensure_ascii=False)
        })

df_threads = pd.DataFrame(records)


In [ ]:
# flatten the jsonl file and export to csv

jsonl_path = r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_threads.jsonl"
out_flat_csv = r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bluesky_comments.csv"

rows = []

def walk(node, root_uri=None, parent_uri=None):
    post = node.get("post")
    if post:
        uri = post.get("uri")
        author_info = post.get("author", {}) or {}
        record = post.get("record", {}) or {}
        viewer = post.get("viewer", {}) or {}

        # Extract embed information (external only)
        embed_data = record.get("embed", {}) or {}
        embed_type = embed_data.get("$type", "") or ""
        embed_uri = embed_title = embed_description = None

        if embed_type == "app.bsky.embed.external":
            external = embed_data.get("external", {}) or {}
            embed_uri = external.get("uri")
            embed_title = external.get("title")
            embed_description = external.get("description")

        # Extract facets (hashtags, links)
        facets = record.get("facets", []) or []
        hashtags, links = [], []

        for facet in facets:
            for feature in (facet.get("features", []) or []):
                feature_type = feature.get("$type", "") or ""
                if feature_type == "app.bsky.richtext.facet#tag":
                    hashtags.append(feature.get("tag"))
                elif feature_type == "app.bsky.richtext.facet#link":
                    links.append(feature.get("uri"))

        rows.append({
            "root_uri": root_uri or uri,
            "parent_uri": parent_uri,
            "uri": uri,
            "cid": post.get("cid"),
            "author_did": author_info.get("did"),
            "author_handle": author_info.get("handle"),
            "author_displayName": author_info.get("displayName"),
            "author_avatar": author_info.get("avatar"),
            "author_createdAt": author_info.get("createdAt"),
            "createdAt": record.get("createdAt"),
            "text": record.get("text"),
            "langs": ",".join(record.get("langs", []) or []),
            "replyCount": post.get("replyCount"),
            "likeCount": post.get("likeCount"),
            "repostCount": post.get("repostCount"),
            "quoteCount": post.get("quoteCount"),
            "bookmarkCount": post.get("bookmarkCount"),
            "indexedAt": post.get("indexedAt"),
            "embed_type": embed_type,
            "embed_uri": embed_uri,
            "embed_title": embed_title,
            "embed_description": embed_description,
            "hashtags": ",".join([h for h in hashtags if h]) if hashtags else None,
            "links": ",".join([l for l in links if l]) if links else None,
            "replyDisabled": viewer.get("replyDisabled"),
            "threadMuted": viewer.get("threadMuted"),
            "has_threadgate": "threadgate" in post,
        })

        root_uri = root_uri or uri
        parent_uri = uri

    for child in (node.get("replies", []) or []):
        walk(child, root_uri=root_uri, parent_uri=parent_uri)

# Load JSONL and flatten
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        thread = obj.get("thread")
        if thread:
            walk(thread)

df_tree = pd.DataFrame(rows)

# keep only replies
df_tree = df_tree[df_tree["root_uri"] != df_tree["uri"]].copy()

# Export flattened CSV with explicit quoting rules
df_tree.to_csv(
    out_flat_csv,
    index=False,
    quoting=csv.QUOTE_ALL,
    escapechar="\\",
    encoding="utf-8"
)

df_tree.shape


(11290, 27)

In [61]:
# filter posts and comments to relevant columns only

# filter posts dataframe
df_10plus_analytic_table = df_10plus[[
    "uri",
    "record.embed.external.title",
    "author.handle",
    "record.createdAt",
    "replyCount",
]].copy()

df_10plus_analytic_table["source"] = "Bluesky"

# rename posts dataframe columns
df_10plus_analytic_table.rename(
    columns={
        "uri": "post_id",
        "record.embed.external.title": "post_title",
        "author.handle": "post_author",
        "record.createdAt": "created_at",
        "replyCount": "comment_count"
    },
    inplace=True
)


# filter comments dataframe
df_tree_analytic_table = df_tree[[
    "root_uri",
    "author_handle",
    "createdAt",
    "text"
]].copy()

# rename posts dataframe columns
df_tree_analytic_table.rename(
    columns={
        "root_uri": "post_id",
        "author_handle": "username",
        "createdAt": "created_at",
        "text": "comment_text",
    },
    inplace=True
)

In [64]:
df_10plus_analytic_table.head(5)

,post_id,post_title,post_author,created_at,comment_count,source
336,at://did:plc:hbpefio3f5csc44msmbgioxz/app.bsky...,NaN,sharonk.bsky.social,2026-02-10T01:44:04.363Z,18,Bluesky
357,at://did:plc:cnpe7qvcyjrhm6w7w7e4atur/app.bsky...,NaN,kevinmkruse.bsky.social,2026-02-10T01:39:29.877Z,45,Bluesky
419,at://did:plc:aunpu65mdrhwfie7ynymlzeh/app.bsky...,NaN,thebulwark.com,2026-02-10T01:23:18.925Z,13,Bluesky
448,at://did:plc:bkmfse64kqf3yjoqdcs65qto/app.bsky...,Justice Department moves to dismiss Steve Bann...,ryanjreilly.com,2026-02-10T01:15:55.417Z,14,Bluesky
588,at://did:plc:maiw2rfujepnphhrejytvuu5/app.bsky...,NaN,qjurecic.bsky.social,2026-02-10T00:44:10.458Z,29,Bluesky


In [63]:
df_tree_analytic_table.head(5)

,post_id,username,created_at,comment_text
4,at://did:plc:kd7pdhsemwqjfacvexf5mglg/app.bsky...,neuseriverhag.bsky.social,2026-02-10T03:10:17.263Z,CW: politics\n\ni really think people underrat...
6,at://did:plc:dsa4uibhvbldyrnkkziijpsg/app.bsky...,voices4justice.bsky.social,2026-02-10T03:13:32.799Z,"""Racist Pig” indeed, says it all‼️"
22,at://did:plc:or5jhlzw46ha45doumorqt6e/app.bsky...,madnuhrah.bsky.social,2026-02-10T03:16:00.858Z,"So, if they block the names of recipients and ..."
23,at://did:plc:or5jhlzw46ha45doumorqt6e/app.bsky...,jonhendry.bsky.social,2026-02-10T05:14:32.454Z,Given the rococo decor of his NY condo maybe i...
24,at://did:plc:or5jhlzw46ha45doumorqt6e/app.bsky...,swfong.bsky.social,2026-02-10T04:48:15.873Z,btw I've seen pictures of the youthful winners...


# Initial EDA Report: Bluesky
## Descriptive Statistics (Bluesky)

In [65]:
# Count comments per root_uri
comment_counts = df_tree['root_uri'].value_counts()

# Filter for roots with >= 10 comments
roots_with_10plus = comment_counts[comment_counts >= 10].index

# Keep only rows where root_uri has >= 10 comments
df_filtered = df_tree[df_tree['root_uri'].isin(roots_with_10plus)]

print(f"Original threads: {df_tree['root_uri'].nunique()}")
print(f"Threads with >=10 comments: {len(roots_with_10plus)}")
print(f"Original rows: {len(df_tree)}")
print(f"Filtered rows: {len(df_filtered)}")

# Optional: Add a column with the comment count for each thread
df_filtered['thread_comment_count'] = df_filtered['root_uri'].map(comment_counts)

# Save the filtered data
df_filtered.to_csv(r"C:\Users\celin\OneDrive\Education\USD-MS-ADS\ADS-509-Applied-Text-Mining\Final-Project\ADS-509_LLM\scripts\bsky_thread_nodes_filtered.csv", index=False)

Original threads: 1913
Threads with >=10 comments: 184
Original rows: 11290
Filtered rows: 6644


C:\Users\celin\AppData\Local\Temp\ipykernel_10744\3299054371.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['thread_comment_count'] = df_filtered['root_uri'].map(comment_counts)


In [24]:
# Count comments per root_uri
comment_counts = df_tree['root_uri'].value_counts()

# Filter for roots with >= 10 comments
roots_with_10plus = comment_counts[comment_counts >= 10].index

# Keep only rows where root_uri has >= 10 comments
df_filtered = df_tree[df_tree['root_uri'].isin(roots_with_10plus)]

print(f"Posts: {df_filtered['root_uri'].nunique()}")
print(f"Comments: {len(df_filtered)}")
print(f"Unique Post Authors: {df_filtered['author_handle'].nunique()}")
print(f"Unique Commenters: {df_filtered['cid'].nunique()}")
print(f"Date Range: {min('createdAt')} to 2026-02-01")

Posts: 0
Comments: 0
Unique Post Authors: 0
Unique Commenters: 0
Date Range: A to 2026-02-01
